# Best Model segun error absoluto de la predicción de 201912

In [803]:
# from google.colab import drive
import pandas as pd
import os


#  Definir rutas en Drive
BASE_INTERMEDIOS   = r'C:\Users\Elisabeth\Desktop\MAESTRIA_AUSTRAL\Labo_III\labo3-2025v\entrega_final\intermedios'
BASE_OUTPUTS   = r'C:\Users\Elisabeth\Desktop\MAESTRIA_AUSTRAL\Labo_III\labo3-2025v\entrega_final\output'

os.makedirs(BASE_OUTPUTS, exist_ok=True)

# Carga de archivos desde Drive
print("🔄 Cargando archivos desde Drive...")
#pred_autogluon = pd.read_csv(os.path.join(BASE_OUTPUTS, "pred_modelo_autogluon.csv"))
pred_autogluon = pd.read_csv(os.path.join(BASE_OUTPUTS, "pred_modelo_Autogluon.csv"))
pred_lgbm = pd.read_csv(os.path.join(BASE_OUTPUTS, "pred_modelo_lgbm_Best.csv"))
pred_RL= pd.read_csv(os.path.join(BASE_OUTPUTS, "pred_modelo_RL.csv"))

error_autogluon = pd.read_csv(os.path.join(BASE_OUTPUTS, "error_modelo_autogluon.csv"))
error_lgbm = pd.read_csv(os.path.join(BASE_OUTPUTS, "error_modelo_lgbm.csv"))
error_RL = pd.read_csv(os.path.join(BASE_OUTPUTS, "error_modelo_RL.csv"))

df_limpio= pd.read_csv(os.path.join(BASE_INTERMEDIOS, "df_limpio_product_id.csv"))

# Renames
pred_autogluon.rename(columns={'tn': 'tn_pred'}, inplace=True)
error_lgbm = error_lgbm.rename(columns={'target': 'tn_real'})  # renombrar 'tn' a 'tn_real'
# seleccionar solo las columnas deseadas
error_lgbm = error_lgbm[['product_id', 'tn_real', 'tn_pred', 'abs_error', 'rel_error']]

🔄 Cargando archivos desde Drive...


### Calculamos el promedio de la tn por product_id de los últimos 12 meses para 201912
Sera otra de las opciones de predicción

In [804]:
# Calcular promedio tn de los últimos 12 meses disponibles por product_id==201912
ultimos_meses = [201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908, 201909, 201910, 201911,201912]
pred_prom_dic19_tn_12M = (
    df_limpio[df_limpio['periodo'].isin(ultimos_meses)]
      .groupby('product_id', as_index=False)
      .agg(avg=('tn','mean'))

)
pred_prom_dic19_tn_12M.head()

,product_id,avg
0,20001,1454.732692
1,20002,1175.437148
2,20003,784.976412
3,20004,627.215330
4,20005,668.270122


### Calculamos el promedio de la tn por product_id de los últimos 12 meses para 201910
Con el objetivo de calcular el error absoluto con la tn real

In [805]:
# Calcular promedio tn de los últimos 12 meses disponibles por product_id==201910
ultimos_meses = [201811, 201812,201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908, 201909, 201910]
pred_prom_oct19_tn_12M = (
    df_limpio[df_limpio['periodo'].isin(ultimos_meses)]
      .groupby('product_id', as_index=False)
      .agg(avg=('tn','mean'))

)

pred_prom_oct19_tn_12M.head()

,product_id,avg
0,20001,1487.869425
1,20002,1197.552089
2,20003,796.305674
3,20004,629.387777
4,20005,638.415252


In [806]:
df_avg=df_limpio[['product_id','tn']][df_limpio['periodo']==201912]#aca lo cambie estaba 201910
df_avg.head()

,product_id,tn
35,20001,1504.68860
71,20002,1087.30860
107,20003,892.50130
143,20004,637.90000
179,20005,593.24445


### Calculamos el error absoluto entre el promedio de los últimos 12 meses para 201912 y la tn real


In [807]:
# Average: Merge entre tn y avg oct 2019
df_avg= pd.merge(df_avg,pred_prom_oct19_tn_12M, on='product_id', how='left')
df_avg.head()

,product_id,tn,avg
0,20001,1504.68860,1487.869425
1,20002,1087.30860,1197.552089
2,20003,892.50130,796.305674
3,20004,637.90000,629.387777
4,20005,593.24445,638.415252


In [808]:
# Calculo de error 201910
df_avg['abs_error_avg'] = (df_avg['tn'] - df_avg['avg']).abs()



In [809]:
df_avg.rename(columns={'avg': 'tn_pred_avg'}, inplace=True)
df_avg.drop(columns=['tn'], inplace=True)
df_avg.head()

,product_id,tn_pred_avg,abs_error_avg
0,20001,1487.869425,16.819175
1,20002,1197.552089,110.243489
2,20003,796.305674,96.195626
3,20004,629.387777,8.512223
4,20005,638.415252,45.170802


### Unimos los errores absolutos de todos los modelos

In [810]:
# Merge por product_id y agregar sufijos para columnas comunes
error_merge = pd.merge(
    error_lgbm,
    error_autogluon,
    on='product_id',
    suffixes=('_lgbm', '_auto')
)
error_merge.head()

,product_id,tn_real_lgbm,tn_pred_lgbm,abs_error_lgbm,rel_error,tn_real_auto,tn_pred_auto,abs_error_auto,pct_error,weight
0,20001,1504.68860,1072.377715,432.310884,0.287309,1504.68856,1607.053592,102.365032,0.068031,0.059840
1,20012,173.13004,334.821354,161.691319,0.933930,173.13004,325.928112,152.798072,0.882562,0.006885
2,20008,195.36855,347.529457,152.160912,0.778840,195.36854,430.347867,234.979327,1.202749,0.007770
3,20014,272.02810,390.193328,118.165221,0.434386,272.02812,441.499765,169.471645,0.622993,0.010818
4,20023,181.13277,283.990366,102.857599,0.567857,181.13277,305.751655,124.618885,0.687997,0.007203


In [811]:
# 1) Renombrar tn_real_lgbm → tn_real
error_merge.rename(columns={'tn_real_lgbm':'tn_real'}, inplace=True)

# 2) Identificar todas las columnas a eliminar
cols_to_drop = [
    'tn_real_auto', 
] + [c for c in error_merge.columns 
     if 'rel_error' in c or 'pct_error' in c]

# 3) Eliminarlas en el mismo DataFrame
error_merge.drop(columns=cols_to_drop, inplace=True)

# 4) Comprobar resultado

error_merge.head()

,product_id,tn_real,tn_pred_lgbm,abs_error_lgbm,tn_pred_auto,abs_error_auto,weight
0,20001,1504.68860,1072.377715,432.310884,1607.053592,102.365032,0.059840
1,20012,173.13004,334.821354,161.691319,325.928112,152.798072,0.006885
2,20008,195.36855,347.529457,152.160912,430.347867,234.979327,0.007770
3,20014,272.02810,390.193328,118.165221,441.499765,169.471645,0.010818
4,20023,181.13277,283.990366,102.857599,305.751655,124.618885,0.007203


In [812]:
error_merge_final = pd.merge(
    error_merge,
    error_RL,
    on='product_id')

error_merge_final.head()

,product_id,tn_real,tn_pred_lgbm,abs_error_lgbm,tn_pred_auto,abs_error_auto,weight,tn_pred_RL,abs_error_RL
0,20001,1504.68860,1072.377715,432.310884,1607.053592,102.365032,0.059840,1260.707460,243.981100
1,20012,173.13004,334.821354,161.691319,325.928112,152.798072,0.006885,295.419673,122.289633
2,20008,195.36855,347.529457,152.160912,430.347867,234.979327,0.007770,281.057816,85.689276
3,20014,272.02810,390.193328,118.165221,441.499765,169.471645,0.010818,322.422187,50.394067
4,20023,181.13277,283.990366,102.857599,305.751655,124.618885,0.007203,234.179244,53.046474


In [813]:
# Partimos de tu df original
df = error_merge_final.copy()

# Lista de columnas de error
error_cols = ['abs_error_lgbm', 'abs_error_auto', 'abs_error_RL']

# 1) Calculamos para cada producto cuál columna tiene el valor mínimo
best_idx = df.set_index('product_id')[error_cols].idxmin(axis=1)

# 2) Extraemos el nombre del modelo a partir del sufijo de la columna
best_model = best_idx.str.replace('abs_error_', '')

# 3) Opcional: también recuperar el valor mínimo de error
min_error = df.set_index('product_id')[error_cols].min(axis=1)

# 4) Construimos el DataFrame resultante
df_best = pd.DataFrame({
    'product_id': best_model.index,
    'best_model': best_model.values,
    'min_abs_error': min_error.values
}).reset_index(drop=True)

print(df_best.head())

   product_id best_model  min_abs_error
0       20001       auto     102.365032
1       20012         RL     122.289633
2       20008         RL      85.689276
3       20014         RL      50.394067
4       20023         RL      53.046474


In [814]:
pred_lgbm.rename(columns={'tn_pred': 'lgbm'}, inplace=True)
pred_autogluon.rename(columns={'tn_pred': 'auto'}, inplace=True)
pred_RL.rename(columns={'pred': 'RL'}, inplace=True)

In [815]:
merge_pred= pd.merge(pred_autogluon, pred_lgbm, on='product_id')
merge_pred= pd.merge(merge_pred, pred_RL, on='product_id')
merge_pred.shape[0]

780

In [816]:
merge_pred.head()

,product_id,auto,lgbm,RL
0,20001,1318.842610,1072.377715,1162.707525
1,20002,1088.935465,1078.858008,1183.640604
2,20003,718.753417,835.263465,684.763931
3,20004,546.314163,585.657639,580.484961
4,20005,529.565386,522.645069,563.560780


In [817]:
merge_pred_final= pd.merge(merge_pred, df_best, on='product_id')
merge_pred_final.shape[0]

780

In [818]:
merge_pred_final.head()

,product_id,auto,lgbm,RL,best_model,min_abs_error
0,20001,1318.842610,1072.377715,1162.707525,auto,102.365032
1,20002,1088.935465,1078.858008,1183.640604,lgbm,8.450586
2,20003,718.753417,835.263465,684.763931,lgbm,57.237817
3,20004,546.314163,585.657639,580.484961,lgbm,52.242386
4,20005,529.565386,522.645069,563.560780,lgbm,70.599377


In [819]:
# Partimos de tu merge_pred_final
df = merge_pred_final.copy()

# 1) Definimos el mapeo entre el valor de best_model y la columna de predicción
col_map = {
    'auto':    'auto',
    'lgbm':    'lgbm',
    'RL':      'RL'
}

# 2) Creamos el nuevo DF con product_id y la predicción del mejor modelo
df_best_pred = pd.DataFrame({
    'product_id': df['product_id'],
    'best_pred':  df.apply(lambda row: row[col_map[row['best_model']]], axis=1)
})

df_best_pred.head()

,product_id,best_pred
0,20001,1318.842610
1,20002,1078.858008
2,20003,835.263465
3,20004,585.657639
4,20005,522.645069


In [820]:
df_best_pred.to_csv(os.path.join(BASE_OUTPUTS, "best_model_product_id.csv"), index=False)

Agregamos Promedio al Merge de RL/Auto/LGBM

In [821]:
df_final_avg = pd.merge(
    error_merge_final,
    df_avg,
    on='product_id')
df_final_avg.rename(columns={'abs_error': 'abs_error_avg'}, inplace=True)
df_final_avg.head()

,product_id,tn_real,tn_pred_lgbm,abs_error_lgbm,tn_pred_auto,abs_error_auto,weight,tn_pred_RL,abs_error_RL,tn_pred_avg,abs_error_avg
0,20001,1504.68860,1072.377715,432.310884,1607.053592,102.365032,0.059840,1260.707460,243.981100,1487.869425,16.819175
1,20012,173.13004,334.821354,161.691319,325.928112,152.798072,0.006885,295.419673,122.289633,343.879977,170.749937
2,20008,195.36855,347.529457,152.160912,430.347867,234.979327,0.007770,281.057816,85.689276,444.378500,249.009950
3,20014,272.02810,390.193328,118.165221,441.499765,169.471645,0.010818,322.422187,50.394067,434.283848,162.255748
4,20023,181.13277,283.990366,102.857599,305.751655,124.618885,0.007203,234.179244,53.046474,287.076334,105.943564


In [822]:
df_final_avg = df_final_avg.merge(
    pred_prom_dic19_tn_12M[['product_id','avg']],  # sólo product_id y avg
    on='product_id',
    how='left'
)
df_final_avg.head()

,product_id,tn_real,tn_pred_lgbm,abs_error_lgbm,tn_pred_auto,abs_error_auto,weight,tn_pred_RL,abs_error_RL,tn_pred_avg,abs_error_avg,avg
0,20001,1504.68860,1072.377715,432.310884,1607.053592,102.365032,0.059840,1260.707460,243.981100,1487.869425,16.819175,1454.732692
1,20012,173.13004,334.821354,161.691319,325.928112,152.798072,0.006885,295.419673,122.289633,343.879977,170.749937,332.228885
2,20008,195.36855,347.529457,152.160912,430.347867,234.979327,0.007770,281.057816,85.689276,444.378500,249.009950,422.048182
3,20014,272.02810,390.193328,118.165221,441.499765,169.471645,0.010818,322.422187,50.394067,434.283848,162.255748,414.705202
4,20023,181.13277,283.990366,102.857599,305.751655,124.618885,0.007203,234.179244,53.046474,287.076334,105.943564,266.901180


In [823]:
# Lista de columnas de error
error_cols = ['abs_error_lgbm', 'abs_error_auto', 'abs_error_RL', 'abs_error_avg']

# 1) Calculamos para cada producto cuál columna tiene el valor mínimo
best_idx_2 = df_final_avg.set_index('product_id')[error_cols].idxmin(axis=1)

# 2) Extraemos el nombre del modelo a partir del sufijo de la columna
best_model_2= best_idx_2.str.replace('abs_error_', '')

# 3) Opcional: también recuperar el valor mínimo de error
min_error_2 = df_final_avg.set_index('product_id')[error_cols].min(axis=1)

# 4) Construimos el DataFrame resultante
df_best_with_avg = pd.DataFrame({
    'product_id': best_model_2.index,
    'best_model': best_model_2.values,
    'min_abs_error': min_error_2.values
}).reset_index(drop=True)

print(df_best_with_avg.head())

   product_id best_model  min_abs_error
0       20001        avg      16.819175
1       20012         RL     122.289633
2       20008         RL      85.689276
3       20014         RL      50.394067
4       20023         RL      53.046474


In [824]:
df_best_with_avg['best_model'].value_counts(normalize=True)

best_model
RL      0.316667
auto    0.256410
lgbm    0.220513
avg     0.206410
Name: proportion, dtype: float64

In [825]:
pred_lgbm.rename(columns={'tn_pred': 'lgbm'}, inplace=True)
pred_autogluon.rename(columns={'tn_pred': 'auto'}, inplace=True)
pred_RL.rename(columns={'pred': 'RL'}, inplace=True)

merge_pred_2= pd.merge(pred_autogluon, pred_lgbm, on='product_id')
merge_pred_2= pd.merge(merge_pred_2, pred_RL, on='product_id')
merge_pred_2= pd.merge(merge_pred_2,pred_prom_dic19_tn_12M, on='product_id')
merge_pred_2.head()
merge_pred_final_2= pd.merge(merge_pred_2, df_best_with_avg, on='product_id')


merge_pred_final_2.head()

,product_id,auto,lgbm,RL,avg,best_model,min_abs_error
0,20001,1318.842610,1072.377715,1162.707525,1454.732692,avg,16.819175
1,20002,1088.935465,1078.858008,1183.640604,1175.437148,lgbm,8.450586
2,20003,718.753417,835.263465,684.763931,784.976412,lgbm,57.237817
3,20004,546.314163,585.657639,580.484961,627.215330,avg,8.512223
4,20005,529.565386,522.645069,563.560780,668.270122,avg,45.170802


In [826]:
# 1) Definimos el mapeo entre el valor de best_model y la columna de predicción
col_map = {
    'auto':    'auto',
    'lgbm':    'lgbm',
    'RL':      'RL',
    'avg':    'avg'
}

In [827]:
# 2) Creamos el nuevo DF con product_id y la predicción del mejor modelo
df_best_pred_2 = pd.DataFrame({
    'product_id': merge_pred_final_2['product_id'],
    'best_pred':  merge_pred_final_2.apply(lambda row: row[col_map[row['best_model']]], axis=1)
})

df_best_pred_2.head()

,product_id,best_pred
0,20001,1454.732692
1,20002,1078.858008
2,20003,835.263465
3,20004,627.215330
4,20005,668.270122


In [828]:

df_best_pred_2.to_csv(os.path.join(BASE_OUTPUTS, "best_model_product_id_with_avg.csv"), index=False)

PREDICCION CON PROMEDIO SIMPLE DE Autogluon /LGBM /RL / Avg 12 M

In [829]:
# Partimos de tu DataFrame original
prediccion_ponderada = merge_pred_final_2[['product_id','auto','lgbm','RL','avg']].copy()

# Calculamos pred = (auto + lgbm + RL + avg*0.98) / 4
prediccion_ponderada['pred'] = (
      prediccion_ponderada['auto']
    + prediccion_ponderada['lgbm']
    + prediccion_ponderada['RL']
    + prediccion_ponderada['avg'] * 1
) / 4

# 2) Eliminamos las columnas originales
prediccion_ponderada = prediccion_ponderada.drop(
    ['auto','lgbm','RL','avg'],
    axis=1
)

# Ahora prediccion_ponderada sólo tiene product_id y pred
print(prediccion_ponderada.head())

   product_id         pred
0       20001  1252.165135
1       20002  1131.717806
2       20003   755.939306
3       20004   584.918023
4       20005   571.010339


In [830]:
prediccion_ponderada.to_csv(os.path.join(BASE_OUTPUTS, "best_model_product_id_pond_final.csv"), index=False)

PREDICCION CON PROMEDIO SIMPLE DE Autogluon /LGBM

In [831]:
# Partimos de tu DataFrame original
prediccion_ponderada_auto_lgbm = merge_pred_final_2[['product_id','auto','lgbm']].copy()

# 1) Creamos la columna 'pred' como la media de las 4 columnas
prediccion_ponderada_auto_lgbm['pred'] = prediccion_ponderada_auto_lgbm[
    ['auto','lgbm']
].mean(axis=1)

# 2) Eliminamos las columnas originales
prediccion_ponderada_auto_lgbm = prediccion_ponderada_auto_lgbm.drop(
    ['auto','lgbm'],
    axis=1
)

In [832]:
# Ahora prediccion_ponderada sólo tiene product_id y pred

print(prediccion_ponderada_auto_lgbm.head())
prediccion_ponderada_auto_lgbm.to_csv(os.path.join(BASE_OUTPUTS, "best_model_product_id_pond_auto_lgbm.csv"), index=False)

   product_id         pred
0       20001  1195.610162
1       20002  1083.896736
2       20003   777.008441
3       20004   565.985901
4       20005   526.105227


PREDICCION CON PROMEDIO SIMPLE DE Autogluon /LGBM /RL 

In [833]:
# Partimos de tu DataFrame original
prediccion_ponderada_auto_lgbm_RL= merge_pred_final_2[['product_id','auto','lgbm','RL']].copy()

# 1) Creamos la columna 'pred' como la media de las 4 columnas
prediccion_ponderada_auto_lgbm_RL['pred'] = prediccion_ponderada_auto_lgbm_RL[['auto','lgbm','RL']
].mean(axis=1)

# 2) Eliminamos las columnas originales
prediccion_ponderada_auto_lgbm_RL = prediccion_ponderada_auto_lgbm_RL.drop(
    ['auto','lgbm','RL'],
    axis=1
)

In [834]:
# Ahora prediccion_ponderada sólo tiene product_id y pred

print(prediccion_ponderada_auto_lgbm_RL.head())
prediccion_ponderada_auto_lgbm_RL.to_csv(os.path.join(BASE_OUTPUTS, "best_model_product_id_pond_final_without_avg.csv"), index=False)

   product_id         pred
0       20001  1184.642617
1       20002  1117.144692
2       20003   746.260271
3       20004   570.818921
4       20005   538.590412


PREDICCION MEDIANA DE LGBM /RL/AUTOGLUON

In [835]:
# Partimos de tu DataFrame original
prediccion_mediana = merge_pred_final_2[['product_id','auto','lgbm','RL']].copy()

# 1) Creamos la columna 'pred' como la mediana de las 3 columnas
prediccion_mediana['pred'] = prediccion_mediana[['auto','lgbm','RL']].median(axis=1)

# 2) Eliminamos las columnas originales
prediccion_mediana.drop(columns=['auto','lgbm','RL'], inplace=True)

print(prediccion_mediana.head())

   product_id         pred
0       20001  1162.707525
1       20002  1088.935465
2       20003   718.753417
3       20004   580.484961
4       20005   529.565386


In [836]:
# Ahora prediccion_ponderada sólo tiene product_id y pred

print(prediccion_mediana.head())
prediccion_mediana.to_csv(os.path.join(BASE_OUTPUTS, "best_model_product_id_mediana_final_without_avg.csv"), index=False)

   product_id         pred
0       20001  1162.707525
1       20002  1088.935465
2       20003   718.753417
3       20004   580.484961
4       20005   529.565386


PREDICCION CON PROMEDIO SIMPLE DE LGBM /RL 

In [561]:
# Partimos de tu DataFrame original
prediccion_ponderada_lgbm_RL= merge_pred_final_2[['product_id','lgbm','RL']].copy()

# 1) Creamos la columna 'pred' como la media de las 4 columnas
prediccion_ponderada_lgbm_RL['pred'] = prediccion_ponderada_lgbm_RL[['lgbm','RL']
].mean(axis=1)

# 2) Eliminamos las columnas originales
prediccion_ponderada_lgbm_RL = prediccion_ponderada_lgbm_RL.drop(
    ['lgbm','RL'],
    axis=1
)

In [562]:
# Ahora prediccion_ponderada sólo tiene product_id y pred

print(prediccion_ponderada_lgbm_RL.head())
prediccion_ponderada_lgbm_RL.to_csv(os.path.join(BASE_OUTPUTS, "best_model_product_id_pond_lgbm_RL.csv"), index=False)

   product_id         pred
0       20001  1117.542620
1       20002  1131.249306
2       20003   760.013698
3       20004   583.071300
4       20005   543.102924


Ensamble “mínima varianza” usando la matriz de covarianzas de los residuos

In [563]:
# Partimos de df_final_avg con columnas abs_error_* y predicciones
df_201910= df_final_avg.copy()
df_201910.drop(columns=['avg'], inplace=True)
df_201910.head()


,product_id,tn_real,tn_pred_lgbm,abs_error_lgbm,tn_pred_auto,abs_error_auto,weight,tn_pred_RL,abs_error_RL,tn_pred_avg,abs_error_avg
0,20001,1504.68860,1072.377715,432.310884,1607.053592,102.365032,0.059840,1260.707460,243.981100,1487.869425,16.819175
1,20012,173.13004,334.821354,161.691319,325.928112,152.798072,0.006885,295.419673,122.289633,343.879977,170.749937
2,20008,195.36855,347.529457,152.160912,430.347867,234.979327,0.007770,281.057816,85.689276,444.378500,249.009950
3,20014,272.02810,390.193328,118.165221,441.499765,169.471645,0.010818,322.422187,50.394067,434.283848,162.255748
4,20023,181.13277,283.990366,102.857599,305.751655,124.618885,0.007203,234.179244,53.046474,287.076334,105.943564


Calcula los residuos de 201910 (predicción – valor real) para cada modelo:

In [564]:
# df201910: DataFrame con columnas ['product_id','pred_auto','pred_lgbm','pred_RL',,'tn_real']
res = df_201910[['tn_pred_auto','tn_pred_lgbm','tn_pred_RL']].values \
      - df_201910['tn_real'].values[:,None]   # shape (n_products,4)

Mide la covarianza de esos residuos y calcula los pesos que minimizan la varianza del ensamblado:

In [565]:
import numpy as np

# 2.1) Matriz de covarianzas entre los residuos
Σ = np.cov(res, rowvar=False)       # 4×4

# 2.2) Pesos mínimos‐varianza: w = Σ⁻¹·1 / (1ᵀ·Σ⁻¹·1)
Σ_inv = np.linalg.pinv(Σ)
ones  = np.ones(Σ.shape[0])
w_mv  = Σ_inv.dot(ones) / (ones.dot(Σ_inv).dot(ones))

print("Pesos MV por modelo:", w_mv)

Pesos MV por modelo: [0.43736106 0.0236026  0.53903634]


Aplica esos pesos a tus predicciones de diciembre-2019:

In [566]:
#Armamos Predicciones 201912
df_201912=merge_pred_final_2.copy()
df_201912.drop(columns=['best_model'], inplace=True)
df_201912.drop(columns=['min_abs_error'], inplace=True)
df_201912.head()

,product_id,auto,lgbm,RL,avg
0,20001,1318.842610,1072.377715,1162.707525,1454.732692
1,20002,1088.935465,1078.858008,1183.640604,1175.437148
2,20003,718.753417,835.263465,684.763931,784.976412
3,20004,546.314163,585.657639,580.484961,627.215330
4,20005,529.565386,522.645069,563.560780,668.270122


In [567]:
# df_201912: DataFrame con ['product_id','pred_auto_12','pred_lgbm_12','pred_RL_12',]
X12 = df_201912[['auto','lgbm','RL']].values
df_201912['pred_mv'] = X12.dot(w_mv)

In [568]:
#Eliminamos las columnas originales
pred_var= df_201912.drop(
    ['auto','lgbm','RL','avg'],
    axis=1
)

In [407]:
#generamos csv

print(pred_var.head())
pred_var.to_csv(os.path.join(BASE_OUTPUTS, "best_model_pred_var.csv"), index=False)

   product_id      pred_mv
0       20001  1286.415334
1       20002  1113.442861
2       20003   682.126438
3       20004   528.501856
4       20005   515.755626


 Metamodelo de regresión lineal con intercepto (“stacking” ligero)

In [408]:
df_201910.head()

,product_id,tn_real,tn_pred_lgbm,abs_error_lgbm,tn_pred_auto,abs_error_auto,tn_pred_RL,abs_error_RL,tn_pred_avg,abs_error_avg
0,20001,1504.68860,1072.377715,432.310884,1313.879416,190.809144,1260.707460,243.981100,1487.869425,16.819175
1,20012,173.13004,334.821354,161.691319,303.362983,130.232943,295.419673,122.289633,343.879977,170.749937
2,20008,195.36855,347.529457,152.160912,385.463395,190.094855,281.057816,85.689276,444.378500,249.009950
3,20014,272.02810,390.193328,118.165221,382.930297,110.902177,322.422187,50.394067,434.283848,162.255748
4,20023,181.13277,283.990366,102.857599,210.637875,29.505105,234.179244,53.046474,287.076334,105.943564


In [409]:
from sklearn.linear_model import LinearRegression

# 1) Preparamos X10, y10 de octubre-2019
X10 = df_201910[['tn_pred_auto','tn_pred_lgbm','tn_pred_RL']].values
y10 = df_201910['tn_real'].values

# 2) Entrenamos regresión CON intercepto
lr = LinearRegression(fit_intercept=True)
lr.fit(X10, y10)

print("Intercept:", lr.intercept_)
print("Pesos LR:", lr.coef_)

# 3) Aplicamos a diciembre-2019
X12 = df_201912[['auto','lgbm','RL',]].values
df_201912['pred_stack'] = lr.predict(X12)

Intercept: -4.250696873122955
Pesos LR: [0.5569508  0.01174252 0.50444625]


In [410]:
df_201912.head()

,product_id,auto,lgbm,RL,avg,pred_mv,pred_stack
0,20001,1313.879416,1072.377715,1162.707525,1454.732692,1286.415334,1326.631367
1,20002,1093.024428,1078.858008,1183.640604,1175.437148,1113.442861,1214.261713
2,20003,686.143588,835.263465,684.763931,784.976412,682.126438,733.132221
3,20004,515.945564,585.657639,580.484961,627.215330,528.501856,582.806158
4,20005,502.793598,522.645069,563.560780,668.270122,515.755626,566.203894


In [411]:
#Eliminamos las columnas originales
pred_stack= df_201912.drop(
    ['auto','lgbm','RL','avg','pred_mv'],
    axis=1
)
#generamos csv

print(pred_stack.head())
pred_stack.to_csv(os.path.join(BASE_OUTPUTS, "best_model_pred_stack.csv"), index=False)

   product_id   pred_stack
0       20001  1326.631367
1       20002  1214.261713
2       20003   733.132221
3       20004   582.806158
4       20005   566.203894
